In [ ]:
# Усов Руслан Николаевич
# Тема преддипломной практики: «Разработка алгоритмов машинного обучения для обнаружения ошибок в текстах (на примере Частного образовательного учреждения высшего образования «Московский университет имени С.Ю. Витте»)»

# Создание нефункционального тестирования - тест скорости инференса

## Импорты

In [11]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import os
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

print(f'Библиотеки ипортированы:')
print('tensorflow', tf.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)

Библиотеки ипортированы:
tensorflow 2.19.0
pandas 2.3.0
numpy 2.1.3


## Конфигурация

In [12]:
MODEL_PATH = "../model/model.keras"
TOKENIZER_PATH = "../model/tokenizer/tokenizer.json"
CSV_PATH = "../data/processed/dataset.csv"
MAX_LEN = 25

## Проверка конфигурации на наличие файлов

In [13]:
# Проверка наличия каждого файла
missing_files = []

if not os.path.isfile(MODEL_PATH):
    missing_files.append(MODEL_PATH)
if not os.path.isfile(TOKENIZER_PATH):
    missing_files.append(TOKENIZER_PATH)
if not os.path.isfile(CSV_PATH):
    missing_files.append(CSV_PATH)

# Вывод результата
if missing_files:
    print("Ошибка! Не найдены следующие файлы:")
    for file in missing_files:
        print(f"   - {file}")
else:
    print(f"Модель импортирована из пути: {MODEL_PATH}")
    print(f"Токенизатор импортирован из пути: {TOKENIZER_PATH}")
    print(f"Датасет импортирован из пути: {CSV_PATH}")
    print(f"Конфигурация гиперпараметра MAX_LEN установлена: {MAX_LEN}")

Модель импортирована из пути: ../model/model.keras
Токенизатор импортирован из пути: ../model/tokenizer/tokenizer.json
Датасет импортирован из пути: ../data/processed/dataset.csv
Конфигурация гиперпараметра MAX_LEN установлена: 25


## Загрузка модели, токенайзера и датасета

In [14]:
# Загрузка модели
# Проверка существования файла перед загрузкой
if os.path.isfile(MODEL_PATH):
    print(f"Файл модели найден: {MODEL_PATH}")
    model = load_model(MODEL_PATH)
    print("Модель успешно загружена.")
else:
    raise FileNotFoundError(f"айл модели не найден по пути: {MODEL_PATH}")

Файл модели найден: ../model/model.keras
Модель успешно загружена.


In [15]:
# Загрузка токенизатора
# Проверка существования файла перед загрузкой токенизатора
if os.path.isfile(TOKENIZER_PATH):
    print(f"Файл токенизатора найден: {TOKENIZER_PATH}")
    with open(TOKENIZER_PATH, "r", encoding="utf-8") as f:
        tokenizer = tokenizer_from_json(f.read())
    print("Токенизатор успешно загружен.")
else:
    raise FileNotFoundError(f"Файл токенизатора не найден по пути: {TOKENIZER_PATH}")

Файл токенизатора найден: ../model/tokenizer/tokenizer.json
Токенизатор успешно загружен.


In [19]:
# Загрузка датасета
# Проверка существования файла
if not os.path.isfile(CSV_PATH):
    raise FileNotFoundError(f"Файл не найден по пути: {CSV_PATH}")
print(f"Файл найден: {CSV_PATH}")

# Загрузка датасета
df = pd.read_csv(CSV_PATH)

# Проверка наличия колонки "noisy"
if "noisy" not in df.columns:
    raise ValueError("В датасете отсутствует колонка 'noisy'.")

# Получение первых 10 'грязных' слов
sample_texts = df["noisy"].astype(str).head(10).tolist()
print("Примеры 'грязных' слов:", sample_texts)

Файл найден: ../data/processed/dataset.csv
Примеры 'грязных' слов: ['Apenines', 'Appenines', 'Athenean', 'Atheneans', 'Bernouilli', 'Blitzkreig', 'Brasillian', 'Britian', 'Brittish', 'Ceasar']


## Токенизация

In [20]:
sequences = tokenizer.texts_to_sequences(sample_texts)
sample_data = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

## Функция latency-теста

In [50]:
# Функция latency-теста
def test_inference_latency(sample_data, model, threshold=0.5):
    start = time.time()
    _ = model.predict(sample_data)
    elapsed = time.time() - start
    if elapsed < threshold:
        print(f"✅ Латентность в пределах нормы: {elapsed:.2f}s")
    else:
        print(f"❌ Латентность {elapsed:.2f}s выше порога {threshold:.2f}s")

# Запуск теста
test_inference_latency(sample_data, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
✅ Латентность в пределах нормы: 0.09s
